# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [1]:
with open("real_estate_sales_data.txt") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

Created a chunk of size 110, which is longer than the specified 100
Created a chunk of size 111, which is longer than the specified 100
Created a chunk of size 107, which is longer than the specified 100
Created a chunk of size 106, which is longer than the specified 100
Created a chunk of size 107, which is longer than the specified 100
Created a chunk of size 106, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 113, which is longer than the specified 100
Created a chunk of size 116, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 109, which is longer than the specified 100
Created a chunk of size 110, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 108, which is longer than the specified 100
Created a chunk of size 114, which is longer tha

In [5]:
docs[0]

Document(page_content='[客户问题] 小米 SU7 的动力性能怎么样？  \n[销售回答] 小米 SU7 的动力性能十分强劲，双电机版最大功率为673马力，百公里加速仅需')

In [6]:
len(docs)

43

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

/opt/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
query = "su7有哪些颜色"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 小米 SU7 的外观颜色有哪些选择？  
[销售回答] 小米 SU7 有海湾蓝、橄榄绿、雅灰、霞光紫、寒武岩灰、流星蓝、熔岩橙、珍珠白、钻石黑共9种车身颜色，您可以根据自己的喜好进行选择。

[客户问题] 小米 SU7 的内饰氛围灯可以调节吗？  
[销售回答] 小米 SU7 的内饰氛围灯可以进行多种颜色和亮度的调节，您可以根据自己的喜好和心情营造出不同的车内氛围。

[客户问题] 小米 SU7 的内饰质量如何？  
[销售回答] 小米 SU7 的内饰采用高品质材料制作，工艺精细，座椅舒适，还有多种内饰颜色可供选择，能给您带来豪华舒适的驾乘感受。

[客户问题] 小米 SU7 的大灯照明效果好吗？  
[销售回答] 小米 SU7 的大灯采用先进的照明技术，亮度高、照射范围广、穿透力强，无论是在夜间行驶还是在恶劣天气条件下，都能为您提供良好的照明效果，保障行车安全。



In [11]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [12]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [13]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10933d750>, search_kwargs={'k': 3})

In [14]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

/opt/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 小米 SU7 的外观颜色有哪些选择？  
[销售回答] 小米 SU7 有海湾蓝、橄榄绿、雅灰、霞光紫、寒武岩灰、流星蓝、熔岩橙、珍珠白、钻石黑共9种车身颜色，您可以根据自己的喜好进行选择。

[客户问题] 小米 SU7 的内饰氛围灯可以调节吗？  
[销售回答] 小米 SU7 的内饰氛围灯可以进行多种颜色和亮度的调节，您可以根据自己的喜好和心情营造出不同的车内氛围。

[客户问题] 小米 SU7 的内饰质量如何？  
[销售回答] 小米 SU7 的内饰采用高品质材料制作，工艺精细，座椅舒适，还有多种内饰颜色可供选择，能给您带来豪华舒适的驾乘感受。



In [15]:
docs = topK_retriever.get_relevant_documents("su7续航有多久")

In [16]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 小米 SU7 的车辆质保政策是怎样的？  
[销售回答] 小米 SU7 提供了较长时间和里程的质保政策，具体的质保内容和期限您可以查阅相关资料或者咨询我们，让您购车后无后顾之忧。

[客户问题] 小米 SU7 的车载导航好用吗？  
[销售回答] 小米 SU7 的车载导航系统精准、智能，能够实时更新路况信息，为您提供最优的行车路线规划，让您的出行更加便捷。

[客户问题] 小米 SU7 的空间大小怎么样？  
[销售回答] 小米 SU7 车长4997mm，轴距3000mm，车内空间宽敞，无论是前排还是后排，都能为乘客提供充足的头部和腿部空间，让您乘坐更加舒适。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [17]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [18]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 小米 SU7 的外观颜色有哪些选择？  
[销售回答] 小米 SU7 有海湾蓝、橄榄绿、雅灰、霞光紫、寒武岩灰、流星蓝、熔岩橙、珍珠白、钻石黑共9种车身颜色，您可以根据自己的喜好进行选择。



### 提取向量数据库中的`销售回答`

In [19]:
docs = retriever.get_relevant_documents(query)

In [20]:
docs[0].page_content

'[客户问题] 小米 SU7 的外观颜色有哪些选择？  \n[销售回答] 小米 SU7 有海湾蓝、橄榄绿、雅灰、霞光紫、寒武岩灰、流星蓝、熔岩橙、珍珠白、钻石黑共9种车身颜色，您可以根据自己的喜好进行选择。'

In [21]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 小米 SU7 的外观颜色有哪些选择？  \n',
 '小米 SU7 有海湾蓝、橄榄绿、雅灰、霞光紫、寒武岩灰、流星蓝、熔岩橙、珍珠白、钻石黑共9种车身颜色，您可以根据自己的喜好进行选择。']

In [22]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [23]:
ans

'小米 SU7 有海湾蓝、橄榄绿、雅灰、霞光紫、寒武岩灰、流星蓝、熔岩橙、珍珠白、钻石黑共9种车身颜色，您可以根据自己的喜好进行选择。'

#### 尝试各种问题

In [24]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [25]:
query = "售后服务如何"

print(sales(query))

['我们有专业的售后服务团队和完善的售后服务体系，为您提供全方位的售后保障，无论是维修、保养还是咨询，都能及时为您解决问题。']


In [26]:
print(sales(query, 0.75))

['我们有专业的售后服务团队和完善的售后服务体系，为您提供全方位的售后保障，无论是维修、保养还是咨询，都能及时为您解决问题。']


In [28]:
query = "颜色有哪些"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

/opt/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []

score:0.75 ans: ['小米 SU7 有海湾蓝、橄榄绿、雅灰、霞光紫、寒武岩灰、流星蓝、熔岩橙、珍珠白、钻石黑共9种车身颜色，您可以根据自己的喜好进行选择。']

score:0.5 ans: ['小米 SU7 有海湾蓝、橄榄绿、雅灰、霞光紫、寒武岩灰、流星蓝、熔岩橙、珍珠白、钻石黑共9种车身颜色，您可以根据自己的喜好进行选择。', '小米 SU7 的内饰氛围灯可以进行多种颜色和亮度的调节，您可以根据自己的喜好和心情营造出不同的车内氛围。', '小米 SU7 的大灯采用先进的照明技术，亮度高、照射范围广、穿透力强，无论是在夜间行驶还是在恶劣天气条件下，都能为您提供良好的照明效果，保障行车安全。', '小米 SU7 的内饰采用高品质材料制作，工艺精细，座椅舒适，还有多种内饰颜色可供选择，能给您带来豪华舒适的驾乘感受。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [30]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [32]:
qa_chain({"query": "su7售价多少钱"})

/opt/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': 'su7售价多少钱', 'result': '对不起，我不知道su7的售价。'}

In [33]:
qa_chain({"query": "隔音效果如何"})

/opt/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '隔音效果如何',
 'result': '对不起，您没有提供足够的信息来回答您的问题。请提供更多的上下文或详细信息，以便我能更好地帮助您。'}

In [ ]:
print(sales("颜色"))

## 加载 FAISS 向量数据库已有结果

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings()

In [35]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [36]:
qa_chain({"query": "我想买别墅，你们有么"})

{'query': '我想买别墅，你们有么',
 'result': '对不起，我无法提供这样的服务。我是一个人工智能，我可以帮助回答问题，但我不能出售商品或房产。'}

In [37]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [38]:
qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能帮助你购买别墅，因为我是一个人工智能，没有提供房地产服务的功能。'}

In [39]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [40]:
result = qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [41]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮你购买别墅。我是一个AI助手，我主要用来提供信息和回答问题。',
 'source_documents': []}